# CLIP Embeddings and NLP prompt

If you run the [Generate Embeddings Workflow](https://docs.aperturedata.io/workflows/embeddings_extraction), 
embeddings will be computed for each image on ApertureDB, using the "ViT-B/16" model from [OpenAI CLIP](https://openai.com/index/clip/).

We can query using a natural language prompt:

In [ ]:
from aperturedb import Connector as Connector
from aperturedb import NotebookHelpers as nh

client = Connector.Connector(host="<DB_HOST>", user="admin", password="<YOUR_PASSWORD_HERE>")

response, _ = client.query([{"GetStatus": {}}])
client.print_last_response()

In [ ]:
import clip

model, preprocess = clip.load("ViT-B/16", device="cpu")

# Natural language prompt
prompt = "a photo of a group of more than 2 people"

search_tokens = clip.tokenize([prompt]).to("cpu")
search_embeddings = model.encode_text(search_tokens)

blobs = search_embeddings[0].detach().numpy().tobytes()

query = [{
    "FindDescriptor": {
        "_ref": 1,
        "k_neighbors": 10,
        "set": "wf_embeddings_clip",
    }
}, {
    "FindImage": {
        "_ref": 2,
        "blobs": True,
        "is_connected_to": {
            "ref": 1
        },
        "results": {
            "limit": 10
        }
    }
}]

r, blobs = client.query(query, [blobs])
client.print_last_response()

print(len(blobs))
nh.display(blobs)